In [54]:
import numpy as np
import pandas as pd
import random
import os
import logging
from typing import List, Tuple, Any, Dict
from tqdm import tqdm
import json
from math import sqrt

from data import loader

# --- Constants ---
HQ_POLYGON = 18
MAX_TIME = 360  # minutes
LOADING_TIME = 30
UNLOADING_TIME = 30
TRUCK_CAPACITY = 8000

In [55]:
# --- Utility Functions ---

def euclidean_distance(p1: List[float], p2: List[float]) -> float:
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def compute_time_matrix(polygon_df: pd.DataFrame) -> pd.DataFrame:
    polygons = polygon_df['Poligono'].tolist()
    coords = polygon_df.set_index('Poligono')[['X', 'Y']].to_dict('index')
    time_data = [
        {
            'origin': i,
            'target': j,
            'time': np.ceil((euclidean_distance(list(coords[i].values()), list(coords[j].values())) / 1000 / 10) * 60)
        }
        for i in polygons for j in polygons
    ]
    time_df = pd.DataFrame(time_data)
    return time_df.pivot(index='origin', columns='target', values='time')


In [56]:
# --- VRP Class ---

class VRP:
    def __init__(self, inventory_df: pd.DataFrame, polygon_df: pd.DataFrame):
        self.inventory_df = inventory_df.copy()
        self.polygon_df = polygon_df
        self.time_df = compute_time_matrix(polygon_df)
        self.reset()

    def reset(self):
        self.current_state = {
            'location': HQ_POLYGON,
            'time': 0,
            'load': 0,
            'inventory': [],
            'delivered': []
        }
        self.action_history = []
        self.remaining = self._get_initial_remaining()

    def _get_initial_remaining(self) -> pd.DataFrame:
        return self.inventory_df.groupby('polygon').agg({'amount': 'sum'}).reset_index()

    def get_state(self) -> dict:
        return self.current_state.copy()

    def get_actions(self, state: dict) -> List[dict]:
        actions = []
        # Load at HQ
        if state['location'] == HQ_POLYGON and state['time'] + LOADING_TIME <= MAX_TIME and not state['inventory']:
            actions.append({'type': 'load'})
        # Deliveries
        for order in state['inventory']:
            polygon = order['polygon']
            travel_time = self.time_df.at[state['location'], polygon]
            total_time = travel_time + UNLOADING_TIME + self.time_df.at[polygon, HQ_POLYGON]
            if state['location'] != polygon and state['time'] + total_time <= MAX_TIME:
                actions.append({'type': 'deliver', 'order': order})
            elif state['location'] == polygon and state['time'] + UNLOADING_TIME + self.time_df.at[polygon, HQ_POLYGON] <= MAX_TIME:
                actions.append({'type': 'deliver', 'order': order})
        # Return to HQ
        if state['location'] != HQ_POLYGON:
            travel_time = self.time_df.at[state['location'], HQ_POLYGON]
            if state['time'] + travel_time <= MAX_TIME:
                actions.append({'type': 'return'})
        return actions

    def protocol(self, actions: List[dict], state: dict) -> dict:
        # Prioritize deliver > load > return
        for action in actions:
            if action['type'] == 'deliver':
                return action
        for action in actions:
            if action['type'] == 'load':
                return action
        for action in actions:
            if action['type'] == 'return':
                return action
        return None

    def apply_action(self, action: dict):
        if action['type'] == 'load':
            self._load_truck()
        elif action['type'] == 'deliver':
            self._deliver_order(action['order'])
        elif action['type'] == 'return':
            self._return_to_hq()

    def _load_truck(self):
        capacity = TRUCK_CAPACITY
        new_inventory = []
        new_remaining = []
        for _, row in self.remaining.iterrows():
            amount = row['amount']
            if amount <= capacity:
                new_inventory.append(row.to_dict())
                capacity -= amount
            else:
                new_remaining.append(row.to_dict())
        self.current_state['inventory'] = new_inventory
        self.remaining = pd.DataFrame(new_remaining)
        self.current_state['load'] = TRUCK_CAPACITY - capacity
        self.current_state['time'] += LOADING_TIME
        self.action_history.append(('load', len(new_inventory)))

    def _deliver_order(self, order: dict):
        state = self.current_state
        travel_time = self.time_df.at[state['location'], order['polygon']]
        state['time'] += travel_time + UNLOADING_TIME
        state['location'] = order['polygon']
        state['inventory'].remove(order)
        state['load'] -= order['amount']
        state['delivered'].append(order)
        self.action_history.append(('deliver', order))

    def _return_to_hq(self):
        state = self.current_state
        travel_time = self.time_df.at[state['location'], HQ_POLYGON]
        state['time'] += travel_time
        state['location'] = HQ_POLYGON
        self.action_history.append(('return',))

    def run(self) -> Tuple[dict, list]:
        while self.current_state['time'] < MAX_TIME:
            actions = self.get_actions(self.current_state)
            if not actions:
                break
            selected_action = self.protocol(actions, self.current_state)
            if not selected_action:
                break
            self.apply_action(selected_action)
        return self.current_state, self.action_history

In [57]:
# --- Warehouse Class ---

class Warehouse:
    """
    Central warehouse that stores inventory by species and day.
    """
    def __init__(self):
        self.daily_inventory = {}  # {day: [{'polygon': x, 'specie': y, 'amount': z}]}
        self.max_capacity = TRUCK_CAPACITY * 2
        self.remaining_capacity = self.max_capacity

    def receive_deliveries(self, day: int, deliveries: List[Tuple[int, int, str, str, float]]):
        if day not in self.daily_inventory:
            self.daily_inventory[day] = []
        total_received = 0
        for _, polygon, specie, _, amount in deliveries:
            self.daily_inventory[day].append({'polygon': polygon, 'specie': specie, 'amount': amount})
            total_received += amount
        self.remaining_capacity -= total_received

    def get_available_orders(self, day: int) -> pd.DataFrame:
        eligible_days = [d for d in self.daily_inventory if d <= day - 3]
        if not eligible_days:
            return pd.DataFrame(columns=['polygon', 'amount'])
        records = []
        for d in eligible_days:
            records.extend(self.daily_inventory[d])
        if not records:
            return pd.DataFrame(columns=['polygon', 'amount'])
        df = pd.DataFrame(records)
        grouped = df.groupby('polygon').agg({'amount': 'sum'}).reset_index()
        return grouped

    def update_after_delivery(self, delivered_orders: List[dict], day: int):
        eligible_days = [d for d in self.daily_inventory if d <= day - 3]
        total_removed = 0
        for delivered in delivered_orders:
            amount_to_remove = delivered['amount']
            for d in eligible_days:
                current = self.daily_inventory[d]
                for i in range(len(current)):
                    entry = current[i]
                    if entry['polygon'] == delivered['polygon'] and entry['specie'] == delivered.get('specie', entry['specie']):
                        stored_amount = entry['amount']
                        if stored_amount <= amount_to_remove:
                            amount_to_remove -= stored_amount
                            total_removed += stored_amount
                            current.pop(i)
                            break
                        else:
                            entry['amount'] -= amount_to_remove
                            total_removed += amount_to_remove
                            amount_to_remove = 0
                            break
                if amount_to_remove == 0:
                    break
        self.remaining_capacity = min(self.remaining_capacity + total_removed, self.max_capacity)
        print(f"[Day {day}] Freed up {total_removed} units. New capacity: {self.remaining_capacity}")

    def is_empty(self):
        return self.max_capacity == self.remaining_capacity


In [58]:
# --- SASCOpt Class ---

class SASCOpt:
    """
    Optimizes supply chain scheduling using Simulated Annealing.
    """
    def __init__(
        self,
        demand_df: pd.DataFrame,
        prices_df: pd.DataFrame,
        polygon_df: pd.DataFrame,
        max_load: int = TRUCK_CAPACITY,
        transport_cost: float = 4500,
        iterations: int = 1000,
        initial_temp: float = 10000,
        cooling_rate: float = 0.995,
        log_file: str = './output/SA_Supply_Chain.log'
    ):
        self.demand_df = demand_df.copy()
        self.prices_df = prices_df.copy()
        self.polygon_df = polygon_df.copy()
        self.max_load = max_load
        self.transport_cost = transport_cost
        self.iterations = iterations
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.log_file = log_file

        self.demands = self._aggregate_demands()
        self.species = self.demands['specie'].unique()
        self.suppliers = self.prices_df['supplier'].unique()

        self._setup_logging()

    def _aggregate_demands(self) -> pd.DataFrame:
        return self.demand_df.groupby(['polygon', 'specie'])['demand'].sum().reset_index()

    def _setup_logging(self):
        if os.path.exists(self.log_file):
            logging.shutdown()
            os.remove(self.log_file)
        logging.basicConfig(filename=self.log_file, level=logging.INFO)

    def initial_solution(self) -> Tuple[List[Tuple[int, Any, Any, Any, float]], List[Dict]]:
        remaining = self.demands.copy()
        order_schedule = []
        vrp_action_history = []  # NEW: Track all VRP actions
        warehouse = Warehouse()
        day = 0

        while (remaining['demand'].sum() > 0) or not (warehouse.is_empty()):
            day += 1
            truck_remaining_capacity = self.max_load
            fulfilled_indices = []
            day_plan = []

            for idx, row in remaining[remaining['demand'] > 0].sample(frac=1).iterrows():
                polygon, specie, demand = row['polygon'], row['specie'], row['demand']
                if (demand > truck_remaining_capacity) or (truck_remaining_capacity + demand > warehouse.remaining_capacity):
                    continue
                valid_suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].unique()
                if len(valid_suppliers) == 0:
                    continue
                supplier = random.choice(valid_suppliers)
                day_plan.append((day, polygon, specie, supplier, demand))
                truck_remaining_capacity -= demand
                fulfilled_indices.append(idx)

            if day_plan:
                order_schedule.extend(day_plan)
                remaining.loc[fulfilled_indices, 'demand'] = 0
                warehouse.receive_deliveries(day, day_plan)

            available_inventory = warehouse.get_available_orders(day)
            vrp_optimizer = VRP(
                inventory_df=available_inventory,
                polygon_df=self.polygon_df
            )
            final_state, action_history = vrp_optimizer.run()

            # NEW: Log VRP actions with the day
            vrp_action_history.append({
                "day": day,
                "actions": action_history
            })

            warehouse.update_after_delivery(final_state['delivered'], day)

        return order_schedule, vrp_action_history


    def fitness(self, schedule: List[Tuple[int, Any, Any, Any, float]]) -> float:
        cost = 0
        horizon_days = max(entry[0] for entry in schedule)
        for day in range(1, horizon_days + 1):
            day_orders = [entry for entry in schedule if entry[0] == day]
            day_suppliers = set()
            for (_, polygon, specie, supplier, amount) in day_orders:
                price_row = self.prices_df[
                    (self.prices_df['specie'] == specie) &
                    (self.prices_df['supplier'] == supplier)
                ]
                unit_price = price_row.iloc[0]['price']
                cost += unit_price * amount
                day_suppliers.add(supplier)
            cost += self.transport_cost * len(day_suppliers)
        return cost

    def neighbor(self, schedule: List[Tuple[int, Any, Any, Any, float]]) -> List[Tuple[int, Any, Any, Any, float]]:
        new_schedule = schedule.copy()
        if len(new_schedule) < 2:
            return new_schedule
        idx = random.randint(0, len(new_schedule) - 1)
        day, polygon, specie, _, amount = new_schedule[idx]
        valid_suppliers = self.prices_df[self.prices_df['specie'] == specie]['supplier'].unique()
        new_supplier = random.choice(valid_suppliers)
        new_schedule[idx] = (day, polygon, specie, new_supplier, amount)
        return new_schedule

    def run(self) -> Tuple[List[Tuple[int, Any, Any, Any, float]], float, str, str]:
        order_schedule, vrp_actions = self.initial_solution()
        current_solution = order_schedule
        current_cost = self.fitness(current_solution)
        best_solution = current_solution
        best_cost = current_cost
        temp = self.initial_temp

        for i in tqdm(range(self.iterations)):
            new_solution = self.neighbor(current_solution)
            new_cost = self.fitness(new_solution)
            accept = new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temp)
            if accept:
                current_solution = new_solution
                current_cost = new_cost
                if new_cost < best_cost:
                    best_solution = new_solution
                    best_cost = new_cost
            temp *= self.cooling_rate
            logging.info(f"Iteration {i+1}: Cost = {current_cost:.2f}, Temp = {temp:.2f}")

        logging.info(f"Best solution found with cost: {best_cost:.2f}")

        # Convert results to JSON
        supply_chain_json = json.dumps([
            {"day": day, "polygon": polygon, "specie": specie, "supplier": supplier, "amount": amount}
            for day, polygon, specie, supplier, amount in best_solution
        ], indent=2)

        vrp_actions_json = json.dumps(vrp_actions, indent=2)

        return best_solution, best_cost, supply_chain_json, vrp_actions_json


In [59]:

demand_df, prices_df, polygon_df = loader.create_datasets()
sc_optimizer = SASCOpt(
    demand_df=demand_df.sample(frac=1),
    prices_df=prices_df,
    polygon_df=polygon_df,
)

schedule, cost, schedule_json, vrp_json = sc_optimizer.run()

[Day 1] Freed up 0 units. New capacity: 8000
[Day 2] Freed up 0 units. New capacity: 8000
[Day 3] Freed up 0 units. New capacity: 8000
[Day 4] Freed up 3341 units. New capacity: 11341
[Day 5] Freed up 2726 units. New capacity: 6079
[Day 6] Freed up 1085 units. New capacity: 7164
[Day 7] Freed up 848 units. New capacity: 8012
[Day 8] Freed up 2815 units. New capacity: 10827
[Day 9] Freed up 3567 units. New capacity: 6412
[Day 10] Freed up 1181 units. New capacity: 7593
[Day 11] Freed up 425 units. New capacity: 8018
[Day 12] Freed up 2046 units. New capacity: 10064
[Day 13] Freed up 2773 units. New capacity: 4905
[Day 14] Freed up 1840 units. New capacity: 6745
[Day 15] Freed up 512 units. New capacity: 7257
[Day 16] Freed up 3200 units. New capacity: 10457
[Day 17] Freed up 3627 units. New capacity: 6691
[Day 18] Freed up 1232 units. New capacity: 7923
[Day 19] Freed up 684 units. New capacity: 8607
[Day 20] Freed up 2178 units. New capacity: 10785
[Day 21] Freed up 3123 units. New cap

100%|██████████| 1000/1000 [00:57<00:00, 17.40it/s]


In [63]:
with open("./scheduling/supply_chain_schedule.json", "w") as f:
    f.write(schedule_json)

with open("./scheduling/vrp_actions.json", "w") as f:
    f.write(vrp_json)